In [1]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *
import hashlib

In [2]:
key_path = key_path
project = project_id
dataset = 'bronze'
table = 'bronze_ticker_data'
table_conca = f'{project}.{dataset}.{table}'

# schema = [
#     {'name': 'dd', 'type': 'STRING'},
#     {'name': 'date', 'type': 'DATE'},
#     {'name': 'ticker', 'type': 'STRING'},
#     {'name': 'adj close', 'type': 'INTEGER'},
#     {'name': 'close', 'type': 'INTEGER'},
#     {'name': 'high', 'type': 'INTEGER'},
#     {'name': 'low', 'type': 'INTEGER'},
#     {'name': 'Open', 'type': 'INTEGER'},
#     {'name': 'Volume', 'type': 'INTEGER'},
# ]

In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [4]:
unique_tickers = bigquery.run_query(
    f"""
    SELECT
        ticker
    FROM sara-carles-keepcoding.bronze.bronze_ticker_info
    """
)
unique_tickers

,ticker
0,NYMTI
1,SIX
2,SBOW
3,HIBB
4,CUBA
...,...
5962,VTRS
5963,ZTS
5964,CTLT
5965,EBS


In [5]:
# Assign an initial date to each ticker.
unique_tickers_initial_date = initial_date_by_ticker(unique_tickers, initial_date='2015-01-01')

In [6]:
# Attempt to fetch the maximum date per ticker from BigQuery; fallback to initial dates if an error occurs.
try:
    max_date_by_ticker = bigquery.run_query(
        f"""
        SELECT
            ticker,
            max(date) as date 
        FROM {table_conca} 
        GROUP BY 
            ticker
        """
    )
    max_date_by_ticker['date'] = pd.to_datetime(max_date_by_ticker['date'])
    max_date_by_ticker['date'] = max_date_by_ticker['date'].dt.tz_localize(None)
    
    # Merge with the DataFrame of unique tickers
    max_date_by_ticker = pd.merge(unique_tickers_initial_date, max_date_by_ticker, how='left', on='ticker')

    # Keep the maximum date between both fields
    max_date_by_ticker['date'] = max_date_by_ticker[['date', 'initial_date']].max(axis=1)

    # Remove the auxiliary 'initial_date' column
    max_date_by_ticker.drop(columns=['initial_date'], inplace=True)

except Exception as e:
    print(e)
    max_date_by_ticker = unique_tickers_initial_date
    print(max_date_by_ticker)

In [7]:
# Perform an incremental update to fetch the latest records
try:
    new_df = fetch_historical_data(max_date_by_ticker, start_date_col='date', interval='1d')
    new_df
except:
    new_df = fetch_historical_data(max_date_by_ticker, start_date_col='initial_date', interval='1d')
    new_df

Datos obtenidos para NYMTI.
Datos obtenidos para SIX.
Datos obtenidos para SBOW.
Datos obtenidos para HIBB.
Datos obtenidos para CUBA.
Datos obtenidos para CVLY.
Datos obtenidos para CATC.
Datos obtenidos para LBAI.
Datos obtenidos para NWLI.
Datos obtenidos para OSG.
Datos obtenidos para ETRN.
Datos obtenidos para EVBG.
Datos obtenidos para MODN.
Datos obtenidos para WIRE.
Datos obtenidos para SLCA.
Datos obtenidos para HRT.
Datos obtenidos para AADR.
Datos obtenidos para AAPB.
Datos obtenidos para AAPD.
Datos obtenidos para AAPU.
Datos obtenidos para AAXJ.
Datos obtenidos para ABCS.
Datos obtenidos para ABLLL.
Datos obtenidos para ABLLW.
Datos obtenidos para ACACW.
Datos obtenidos para ACWI.
Datos obtenidos para ACWX.
Error al obtener datos para AFJKR: 'timestamp'
Datos obtenidos para AGIX.
Datos obtenidos para AGMI.
Datos obtenidos para AGNG.
Datos obtenidos para AGZD.
Datos obtenidos para AIA.
Datos obtenidos para AIPI.
Datos obtenidos para AIQ.
Datos obtenidos para AIRL.
Datos obt

In [8]:
# Fields that make up the ID
id_fields = ['ticker', 'date']

# Apply the function to the DataFrame to create the 'id' column
new_df['id'] = new_df.apply(generate_id, axis=1, fields=id_fields)

new_df

,date,open,high,low,close,adjclose,volume,ticker,id
0,2024-07-02,25.000000,25.100000,24.850000,25.059999,25.059999,133500.0,NYMTI,dfbe22f0af0c4e64f5003d511d80e81f
1,2024-07-03,25.000000,25.030001,24.975000,25.016001,25.016001,25600.0,NYMTI,57cb8e05c112d1e2b88a6f9b69f01ddf
2,2024-07-05,25.000000,25.200001,25.000000,25.090000,25.090000,43700.0,NYMTI,33406a44057b0fc22775d896fc734156
3,2024-07-08,25.100000,25.100000,25.000000,25.030001,25.030001,41300.0,NYMTI,2639e4d4958af6795e5110f3d06c70a2
4,2024-07-09,25.030001,25.030001,24.990000,24.990000,24.990000,16600.0,NYMTI,e0feb785202f4c470914a57fdf320394
...,...,...,...,...,...,...,...,...,...
7953284,2024-08-20,69.070000,69.500000,68.669998,69.320000,69.320000,156900.0,PBH,fc44f4ff7f6f74277b3ec497f7f9a973
7953285,2024-08-21,69.529999,70.260002,69.029999,70.150002,70.150002,209800.0,PBH,f981a4cbb399852f54922e07f82c4262
7953286,2024-08-22,70.099998,70.959999,69.800003,70.900002,70.900002,180100.0,PBH,8c92f209388f608ad0727761e7f6c058
7953287,2024-08-23,70.889999,72.190002,70.650002,71.739998,71.739998,176900.0,PBH,f096b502430679001d24402f8b92756d


In [9]:
# En el caso de tener un df en Bigquery, lo leemos y guardamos solo los nuevos registros
try:
    # Filtramos solamente los nuevos registros
    df_incremental = bigquery.select_for_incremental(id='id', table=table_conca, new_df=new_df)

    # Guardamos los datos en bigquery
    bigquery.save_dataframe(df_incremental, project, dataset, table, if_exists='append', schema=None)

# En el caso de no tener datos en Bigquery, guardamos todo el df
except:
    bigquery.save_dataframe(new_df, project, dataset, table, if_exists='append', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
